# Mineração de Dados: Classificação

**Grupo**:

- Gabriel Oliveira Moreira Faria
- Vinícius Oliveira

**Objetivo**:

O objetivo deste trabalho é criar classificadores que buscam prever a aprovação (ou reprovação) de discentes que participaram de um curso online.
Essas bases de dados apresentam características de pessoas que participaram do curso online "Prevenção do uso de drogas".


Os dados estão disponíveis na pasta do drive compartilhada aqui e são formados por 3 arquivos:
"trabalho5_dados_sociais_ID.csv" - dados sócio-econômicos e de perguntas iniciais respondidas pelos participantes,
"trabalho5_dados_modulo1_ID.csv" - dados de acesso ao sistema de ensino durante as atividades referentes ao primeiro módulo do curso, e
"trabalho5_dados_ateh_modulo2_ID.csv" - dados de acesso ao sistema de ensino para as atividades até o segundo módulo do curso.


Nesse trabalho geramos um modelo para cada um das 3 situações a seguir:
1) um modelo que considera apenas as características sócio-econômicos e de perguntas iniciais;
2) um modelo que considera apenas as características sócio-econômicos e de perguntas iniciais, e de acesso considerando o primeiro módulo do curso; e
3) um modelo que considera todos os dados disponíveis.
Para cada uma das 3 situações apresentadas, deve-se gerar ao menos 2 modelos de tipos distintos.

**Com a importância de analisar os parâmetros que fornecemos para os modelos**

In [2]:
import itertools as it
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyclustertend
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import DBSCAN, OPTICS, AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestCentroid
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale

## Leitura dos dados e Análise exploratória

Primeiro, carregaremos os dados e analisaremos as características sócio-econômicas, perguntas e dados do sistem de ensino providos a partir da base de dados.


In [3]:
df = pd.read_csv("data/trabalho5_dados_sociais_4.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 80 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0   id                                                                              1000 non-null   int64 
 1   idade                                                                           1000 non-null   int64 
 2   sexo                                                                            1000 non-null   object
 3   escolaridade                                                                    1000 non-null   object
 4   estadocivil                                                                     1000 non-null   object
 5   ocupacao                                                                        1000 non-null   object
 6   tempodeservico           

In [4]:
df = pd.read_csv("data/trabalho5_dados_modulo1_4.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   quesm1      1000 non-null   int64 
 2   quesm1r     1000 non-null   int64 
 3   forum1      1000 non-null   int64 
 4   forum2      1000 non-null   int64 
 5   forum3      1000 non-null   int64 
 6   forum4      1000 non-null   int64 
 7   ativcolm1   1000 non-null   int64 
 8   ativcolm1r  1000 non-null   int64 
 9   forum1r     1000 non-null   int64 
 10  aprovado    1000 non-null   object
dtypes: int64(10), object(1)
memory usage: 86.1+ KB


In [5]:
df = pd.read_csv("data/trabalho5_dados_ateh_modulo2_4.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   quesm2      1000 non-null   int64 
 2   ativcolm2   1000 non-null   int64 
 3   forum5      1000 non-null   int64 
 4   forum6      1000 non-null   int64 
 5   forum7      1000 non-null   int64 
 6   ativcolm2r  1000 non-null   int64 
 7   quesm2r     1000 non-null   int64 
 8   forum8      1000 non-null   int64 
 9   forum2r     1000 non-null   int64 
 10  aprovado    1000 non-null   object
 11  quesm1      1000 non-null   int64 
 12  quesm1r     1000 non-null   int64 
 13  forum1      1000 non-null   int64 
 14  forum2      1000 non-null   int64 
 15  forum3      1000 non-null   int64 
 16  forum4      1000 non-null   int64 
 17  ativcolm1   1000 non-null   int64 
 18  ativcolm1r  1000 non-null   int64 
 19  forum1r     1000 non-null   int64 
dtypes: int64(